# TC-CLIP Inference Demo for Custom Videos

## Set model path, custom video path, class names.

In [28]:
### Change to your settings ###
output="workspace/inference"
tc_clip_model_path = "zero_shot_k400_llm_tc_clip.pth"   # Your pretrained model saved path
class_names = ntu60_classes = [
    'drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop',
    'pickup', 'throw', 'sitting down', 'standing up (from sitting position)', 
    'clapping', 'reading', 'writing', 'tear up paper', 'wear jacket', 'take off jacket',
    'wear a shoe', 'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap',
    'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 'put down something',
    'reach into pocket', 'hopping (one foot jumping)', 'jump up', 'make a phone call', 
    'playing with phone/tablet', 'typing on a keyboard', 'pointing to something with finger',
    'taking a selfie', 'check time (from watch)', 'rub two hands together', 'nod head/bow',
    'shake head', 'wipe face', 'salute', 'put the palms together', 'cross hands in front',
    'sneeze/cough', 'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)',
    'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition',
    'use a fan (cooling with self)', 'punching/slapping other person', 'kicking other person',
    'pushing other person', 'pat on back of other person', 'point finger at the other person',
    'hugging other person', 'giving something to other person', 'touch other person\'s pocket',
    'handshaking', 'walking towards each other', 'walking apart from each other'
]
# Class names
video_path = "/root/tc-clip/S001C001P001R002A051_rgb.avi" # Custom video path

## No need to change below codes, just run the cells.

In [29]:
# Import libraries
import os
from hydra import initialize, compose
from omegaconf import OmegaConf
from pathlib import Path
import torch

from datasets.pipeline import Compose
from trainers.build_trainer import returnCLIP
from utils.logger import create_logger
from utils.print_utils import colorstr
from utils.tools import load_checkpoint

### Init configs and logger

In [30]:
# Initialize Hydra configs
overrides = [
    f"output={output}",
    "eval=test",
    "trainer=tc_clip",
    f"resume={tc_clip_model_path}"
]

# Initialize Hydra with config path
with initialize(version_base=None, config_path="configs"):
    config = compose(config_name="zero_shot.yaml", overrides=overrides)

In [31]:
# Init settings
OmegaConf.set_struct(config, False)  # Needed to add fields at runtime below

# Define working dir
Path(config.output).mkdir(parents=True, exist_ok=True)

# Logger
logger = create_logger(output_dir=config.output, dist_rank=0, name=f"{config.trainer_name}")
logger.info(f"working dir: {config.output}")

# Whether to use pytorch or apex amp
major, minor = int(torch.__version__.split('.')[0]), int(torch.__version__.split('.')[1])
config.use_torch_amp = (major >= 2)

[12-23 22:18:09 TCCLIP](3979588942.py 9): INFO working dir: workspace/inference


### Build model & load checkpoint

In [32]:
# Build model
model = returnCLIP(config, logger, class_names)
model.cuda()

[12-23 22:18:09 TCCLIP](build_trainer.py 56): INFO Loading CLIP (backbone: ViT-B/16)
Using spatial positional embedding
Weights not found for some missing keys:  ['visual.transformer.resblocks.1.attn.local_global_bias_table', 'visual.transformer.resblocks.2.attn.local_global_bias_table', 'visual.transformer.resblocks.3.attn.local_global_bias_table', 'visual.transformer.resblocks.4.attn.local_global_bias_table', 'visual.transformer.resblocks.5.attn.local_global_bias_table', 'visual.transformer.resblocks.6.attn.local_global_bias_table', 'visual.transformer.resblocks.7.attn.local_global_bias_table', 'visual.transformer.resblocks.8.attn.local_global_bias_table', 'visual.transformer.resblocks.9.attn.local_global_bias_table', 'visual.transformer.resblocks.10.attn.local_global_bias_table', 'visual.transformer.resblocks.11.attn.local_global_bias_table']
[12-23 22:18:10 TCCLIP](build_trainer.py 59): INFO Building TCCLIP
[12-23 22:18:10 TCCLIP](tc_clip_prompt_learner.py 54): INFO Video-condition

TCCLIP(
  (prompt_learner): VPPromptLearner()
  (image_encoder): TCVisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): TCAttentionBlock(
          (attn): TCAttention(
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,

In [33]:
# Load checkpoint
if config.resume:
    epoch_loaded, max_accuray_loaded = load_checkpoint(config, model, None, None, logger, model_only=True)
    logger.info(
            f"Loaded checkpoint at epoch {epoch_loaded} with max accuracy {max_accuray_loaded:.1f}")

[12-23 22:18:10 TCCLIP](tools.py 180): INFO ==============> Resuming from zero_shot_k400_llm_tc_clip.pth....................
[12-23 22:18:11 TCCLIP](tools.py 208): INFO resume model: _IncompatibleKeys(missing_keys=['prompt_learner.token_prefix', 'prompt_learner.token_suffix'], unexpected_keys=[])
[12-23 22:18:11 TCCLIP](tools.py 218): INFO => loaded successfully 'zero_shot_k400_llm_tc_clip.pth' (epoch 9)
[12-23 22:18:11 TCCLIP](1028797808.py 4): INFO Loaded checkpoint at epoch 10 with max accuracy 82.1


### Video preprocessing pipeline

In [34]:
# Video preprocessing pipeline

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)

scale_resize = int(256 / 224 * config.input_size)
collect_keys = ['imgs']

val_pipeline = [
    dict(type='DecordInit'),
    dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.num_frames, test_mode=True),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, scale_resize)),
    dict(type='CenterCrop', crop_size=config.input_size),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCHW'),
    dict(type='Collect', keys=collect_keys, meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
if config.num_crop == 3:
    val_pipeline[3] = dict(type='Resize', scale=(-1, config.input_size))
    val_pipeline[4] = dict(type='ThreeCrop', crop_size=config.input_size)
if config.num_clip > 1:
    val_pipeline[1] = dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.num_frames,
                           multiview=config.num_clip)
val_pipeline = [p for p in val_pipeline if p is not None]

pipeline = Compose(val_pipeline)

dict_file = {'filename': video_path, 'tar': False, 'modality': 'RGB', 'start_index': 0}

### TC-CLIP inference

In [35]:
video = pipeline(dict_file)
video_tensor = video['imgs'].unsqueeze(0).cuda().float() # Size: [1, T, 3, H, W]

# Inference with TC-CLIP
with torch.no_grad():
    if config.use_torch_amp:
        with torch.cuda.amp.autocast():
            output = model(video_tensor)
    else:
        output = model(video_tensor)
    
    logits = output['logits']

pred_index = logits.argmax(-1)

/tmp/ipykernel_155905/164853365.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [36]:
print(f'Logits: {logits}')
print(f'Predicted action category is "{class_names[pred_index]}"')

Logits: tensor([[23.7031, 23.6562, 23.3594, 23.2656, 24.0625, 24.2812, 25.0156, 24.7188,
         26.7188, 23.8438, 23.3281, 23.8438, 23.0469, 23.8906, 24.6875, 25.9688,
         26.3125, 24.3281, 24.8281, 22.6406, 23.5625, 24.7969, 22.9375, 27.2969,
         23.6562, 25.0938, 26.8438, 25.8750, 23.4844, 23.8594, 24.3750, 24.1250,
         22.2812, 24.6562, 23.5469, 24.3594, 25.2031, 23.2500, 23.6094, 24.1250,
         25.1094, 24.7344, 24.4688, 24.6094, 23.3594, 23.2969, 24.5469, 23.7656,
         23.9062, 23.1250, 25.3438, 29.1719, 24.6406, 24.1250, 24.2344, 22.8906,
         24.4062, 24.5469, 25.6875, 26.8438, 25.5625]], device='cuda:0',
       dtype=torch.float16)
Predicted action category is "kicking other person"


Acknowledgements: [ViFi-CLIP's repository](https://github.com/muzairkhattak/ViFi-CLIP).